In [1]:
# Cell 1: Imports & Configuration  
import os, glob  
import numpy as np  
import matplotlib.pyplot as plt  
import cv2  
import tifffile as tiff  
from scipy.signal import find_peaks  
from skimage.filters import threshold_otsu  
from skimage.morphology import closing, opening, disk, remove_small_holes  
%matplotlib inline

In [ ]:
# Paths  
BASE_PATH = '/net/fs-2/scale/OrionStore/Home/mubut0522/repos/hsi'  
OUT_DIR   = '/net/fs-2/scale/OrionStore/Home/mubut0522/repos/prepocessing/processed_cubes'  
FIG_DIR   = '/net/fs-2/scale/OrionStore/Home/mubut0522/repos/prepocessing/figures'  
os.makedirs(OUT_DIR, exist_ok=True)  
os.makedirs(FIG_DIR, exist_ok=True)  

# Save format choice  
SAVE_FORMAT = 'npz'  # options: 'npz', 'mat', 'tiff'  
MIN_WIDTH = 20   # drop any block narrower than this (in pixels)
# Species list for batch processing  
# Species list for batch processing  
species_list = ['abachi','afromasia','ipe','iroko','merbau',  
                'ovangol','padauk','sapelimahonki','tiiki']  
print(len(species_list), "species to process.")

10 species to process.


In [3]:
# Cell 2: Loading & Normalization  
from spectral import open_image  

def load_and_normalize(species_name, white_id='White_swir_.1-4', dark_id='dark-end_swir_.1-27', eps=1e-6):  
    """Load HSI (.hdr/.raw) and apply white/dark reflectance normalization."""  
    hdr_path = glob.glob(os.path.join(BASE_PATH, f"{species_name}_swir_*.hdr"))[0]  
    raw_cube = open_image(hdr_path).load().astype(np.float32)  
    white_cube = open_image(os.path.join(BASE_PATH, f"{white_id}.hdr")).load().astype(np.float32)  
    dark_cube  = open_image(os.path.join(BASE_PATH, f"{dark_id}.hdr")).load().astype(np.float32)  
    white_spec = white_cube.mean((0,1))  
    dark_spec  = dark_cube.mean((0,1))  
    norm_cube  = (raw_cube - dark_spec) / (white_spec - dark_spec + eps)  
    norm_cube  = np.clip(norm_cube, 0, 1)  
    return raw_cube, norm_cube 

In [4]:
# %% 
# Cell 3: Combined Histogram, ROI Mask, and ROI Image

import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.morphology import closing, remove_small_holes, opening, disk

def crop_wood_roi_v2(norm_cube, species_name,
                     disk_close=15, hole_area=20000, disk_open=5, min_area=5000):
    """Crop ROI and save a combined figure of histogram, mask, and ROI."""
    I = norm_cube.mean(axis=2)
    otsu_t = threshold_otsu(I)

    # build mask
    mask = closing(I > otsu_t, disk(disk_close))
    mask = remove_small_holes(mask, area_threshold=hole_area)
    mask = opening(mask, disk(disk_open))

    # keep largest component
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask.astype(np.uint8))
    areas = stats[1:, cv2.CC_STAT_AREA]; largest = np.argmax(areas)+1
    fg = (labels == largest).astype(np.uint8)
    fg = cv2.dilate(fg, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,21)))

    # crop
    ys, xs = np.where(fg); y0,y1,x0,x1 = ys.min(), ys.max(), xs.min(), xs.max()
    roi = norm_cube[y0:y1+1, x0:x1+1, :]

    # Combined figure
    fig, axs = plt.subplots(1,3,figsize=(12,4))
    # histogram
    axs[0].hist(I.ravel(), bins=50, color='gray')
    axs[0].set_title("Intensity Histogram")
    # mask
    axs[1].imshow(fg, cmap='gray')
    axs[1].set_title("Foreground Mask")
    # roi
    axs[2].imshow(roi.mean(axis=2), cmap='gray')
    axs[2].set_title("Cropped ROI")
    for ax in axs: ax.axis('off')
    fig.tight_layout()
    fig.savefig(os.path.join(FIG_DIR, f"{species_name}_crop_summary.png"))
    plt.close(fig)

    return roi


In [5]:
# %% 
# Cell 4: Split Blocks + Single Figure

import matplotlib.pyplot as plt
from scipy.signal import find_peaks

def split_blocks_v2(roi, species_name, expected=3, min_dist=20, prom=0.02):
    """Split ROI and save projection + ROI split figure."""
    W = roi.shape[1]
    proj = roi.mean(axis=(0,2))
    inv = 1 - (proj - proj.min())/(proj.max()-proj.min())
    peaks, props = find_peaks(inv, distance=W//expected-min_dist, prominence=prom)
    # top valleys
    if len(peaks) < expected-1: raise RuntimeError(...)
    top  = np.sort(peaks[np.argsort(props['prominences'])[::-1][:expected-1]])
    # boundaries
    bounds = [0] + top.tolist() + [W]

    # figure
    fig, axs = plt.subplots(2,1,figsize=(6,6),
                             gridspec_kw={'height_ratios':[1,3]})
    axs[0].plot(proj); axs[0].set_title("Width-wise Projection")
    for p in top: axs[0].axvline(p, color='r', linestyle='--')
    axs[1].imshow(roi.mean(axis=2), cmap='gray'); axs[1].set_title("ROI with Splits")
    for p in top: axs[1].axvline(p, color='r', linestyle='--')
    for ax in axs: ax.axis('off')
    fig.tight_layout()
    fig.savefig(os.path.join(FIG_DIR, f"{species_name}_split_summary.png"))
    plt.close(fig)

    # return blocks
    return [roi[:, int(bounds[i]):int(bounds[i+1]), :] 
            for i in range(len(bounds)-1)]


In [6]:
# %% 
# Cell 5: Trim Blocks + Combined Preview

import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.morphology import closing, remove_small_holes, opening, disk

def trim_blocks_v2(raw_blocks, species_name,
                   hole_area=2000, close_disk=7, open_disk=3):
    """Trim each block and save one figure showing all trimmed blocks."""
    trimmed = []
    n = len(raw_blocks)
    fig, axs = plt.subplots(1, n, figsize=(4*n,4))
    for i, b in enumerate(raw_blocks):
        # mask & bbox
        I = b.mean(axis=2)
        t = threshold_otsu(I)
        m = opening(remove_small_holes(closing(I>t, disk(close_disk)), hole_area), disk(open_disk))
        ys, xs = np.where(m)
        if ys.size:
            y0,y1 = ys.min(), ys.max(); x0,x1 = xs.min(), xs.max()
            b = b[y0:y1+1, x0:x1+1, :]
        trimmed.append(b)
        # subplot
        axs[i].imshow(b.mean(axis=2), cmap='gray')
        axs[i].set_title(f"Trimmed {i}\n{b.shape[:2]}")
        axs[i].axis('off')
    fig.tight_layout()
    fig.savefig(os.path.join(FIG_DIR, f"{species_name}_trimmed_blocks.png"))
    plt.close(fig)
    return trimmed


In [7]:
# %%  
# Cell 6: Batch Process, Filter Tiny Splits, & Save  


for species_name in species_list:  
    print(f"\n🔄 Processing {species_name}...")  

    # Load & normalize  
    raw, norm = load_and_normalize(species_name)  

    # Crop ROI + summary fig  
    roi = crop_wood_roi_v2(norm, species_name)  

    # Split ROI into however-many blocks + summary fig  
    raw_blocks = split_blocks_v2(roi, species_name)  

    # Trim each block + save trimmed summary fig  
    blocks     = trim_blocks_v2(raw_blocks, species_name)  

    # Filter out tiny-width blocks  
    kept_blocks = []  
    for i, b in enumerate(blocks):  
        h, w, _ = b.shape  
        if w < MIN_WIDTH:  
            print(f" • Skipping block {i} (width {w} < {MIN_WIDTH})")  
        else:  
            kept_blocks.append((i, b))  

    # Save the remaining blocks  
    for i, b in kept_blocks:  
        fname = f"{species_name}_block{i}.{SAVE_FORMAT}"  
        path  = os.path.join(OUT_DIR, fname)  

        if SAVE_FORMAT == 'npz':  
            np.savez_compressed(path, block=b)  
        elif SAVE_FORMAT == 'mat':  
            from scipy.io import savemat  
            savemat(path, {'block': b})  
        else:  
            tiff.imwrite(path, np.transpose(b, (2, 0, 1)))  

    print(f"✅ Saved {len(kept_blocks)} blocks for {species_name} (filtered out {len(blocks)-len(kept_blocks)})")  



🔄 Processing abachi...


/mnt/users/mubut0522/miniconda3/envs/hsi_torch/lib/python3.10/site-packages/spectral/io/envi.py:187: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


✅ Saved 3 blocks for abachi (filtered out 0)

🔄 Processing afromasia...
 • Skipping block 2 (width 3 < 20)
✅ Saved 2 blocks for afromasia (filtered out 1)

🔄 Processing ipe...
✅ Saved 3 blocks for ipe (filtered out 0)

🔄 Processing iroko...
✅ Saved 3 blocks for iroko (filtered out 0)

🔄 Processing khayamahonki...
 • Skipping block 2 (width 4 < 20)
✅ Saved 2 blocks for khayamahonki (filtered out 1)

🔄 Processing merbau...
✅ Saved 3 blocks for merbau (filtered out 0)

🔄 Processing ovangol...
✅ Saved 3 blocks for ovangol (filtered out 0)

🔄 Processing padauk...
✅ Saved 3 blocks for padauk (filtered out 0)

🔄 Processing sapelimahonki...
✅ Saved 3 blocks for sapelimahonki (filtered out 0)

🔄 Processing tiiki...
✅ Saved 3 blocks for tiiki (filtered out 0)
